## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,East London,ZA,-33.0153,27.9116,66.88,87,100,6.58,light rain
1,1,Severo-Kurilsk,RU,50.6789,156.1250,28.83,75,91,26.98,overcast clouds
2,2,Warmbad,NaN,-28.4500,18.7333,60.84,63,0,9.31,clear sky
3,3,Mogadishu,SO,2.0371,45.3438,78.73,85,100,16.91,overcast clouds
4,4,Chokurdakh,RU,70.6333,147.9167,-23.78,99,100,5.97,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,East London,ZA,-33.0153,27.9116,66.88,87,100,6.58,light rain
2,2,Warmbad,NaN,-28.4500,18.7333,60.84,63,0,9.31,clear sky
3,3,Mogadishu,SO,2.0371,45.3438,78.73,85,100,16.91,overcast clouds
8,8,Busselton,AU,-33.6500,115.3333,74.93,47,98,6.62,overcast clouds
10,10,Eldorado,BR,-23.7869,-54.2836,86.09,45,2,5.97,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                330
City                   330
Country                324
Lat                    330
Lng                    330
Max Temp               330
Humidity               330
Cloudiness             330
Wind Speed             330
Current Description    330
dtype: int64

In [10]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df.dropna()
preferred_cities_df.count()

City_ID                324
City                   324
Country                324
Lat                    324
Lng                    324
Max Temp               324
Humidity               324
Cloudiness             324
Wind Speed             324
Current Description    324
dtype: int64

In [11]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,East London,ZA,66.88,light rain,-33.0153,27.9116,
3,Mogadishu,SO,78.73,overcast clouds,2.0371,45.3438,
8,Busselton,AU,74.93,overcast clouds,-33.6500,115.3333,
10,Eldorado,BR,86.09,clear sky,-23.7869,-54.2836,
11,Mar Del Plata,AR,66.09,light rain,-38.0023,-57.5575,
13,George Town,MY,78.89,few clouds,5.4112,100.3354,
15,Hilo,US,71.80,light rain,19.7297,-155.0900,
16,Jurado,CO,75.47,moderate rain,7.1053,-77.7641,
20,Ribeira Grande,PT,60.66,few clouds,38.5167,-28.7000,
22,Castro,BR,61.36,overcast clouds,-24.7911,-50.0119,


In [12]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    params['location'] = f'{lat},{lng}'
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:b
        hotel_df.loc[index, 'Hotel Name'] = hotels['results'][0]['name']
    except (IndexError):
        print('Hotel not found... skipping.')


Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [15]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.count()

City                   324
Country                324
Max Temp               324
Current Description    324
Lat                    324
Lng                    324
Hotel Name             324
dtype: int64

In [18]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [25]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [26]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))